In [18]:
# This package is required for Watson Discovery 
# You might need to run the command below once if you don't have this package installed.
#!pip install --upgrade ibm-watson

In [19]:
import json

from ibm_watson import DiscoveryV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# WML python SDK
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes, DecodingMethods

## 1. Watson Discovery Configuration

In [20]:
authenticator = IAMAuthenticator('rF5cc4r9ZyEEKDWw7fv9eVvTK8izlkzdQU5R4nULSAv9')
discovery = DiscoveryV2(
    version='2020-08-30',
    authenticator=authenticator
)

discovery.set_service_url('https://api.us-south.discovery.watson.cloud.ibm.com/instances/9b45b9ef-5a76-4921-91cd-8de6226d87e5')

## 2. Watson Discovery Search 

In [21]:
question = 'I’m interested in IBM’s effect on the environment.  What efforts have they been making in sustainability??'

In [22]:
response = discovery.query(
  project_id='aa55396e-690c-4faa-8025-d1ccde561640',
  collection_ids = ['50482a47-2d12-941b-0000-018ab47e2892'],
  passages = {'enabled': True, 
              'max_per_document': 5,
             'find_answers': True},
  natural_language_query = question
).get_result()

with open('data.json', 'w') as f:
    json.dump(response, f)

In [23]:
# inspecting the key fields in the WD output
response.keys()

dict_keys(['matching_results', 'retrieval_details', 'aggregations', 'results'])

In [24]:
# only one relevant document (because one document in the collection)
len(response['results'])

1

In [25]:
# removing some tags
passages = response['results'][0]['document_passages']
passages = [p['passage_text'].replace('<em>', '').replace('</em>', '').replace('\n','') for p in passages]
passages

['Environmental, equitable, and ethical impactBoth internally, and in our work with clients, IBM aspires to make a lasting, positive impact on the world by protecting the environment, advocating for inclusion, and fostering trust and transparency in both technology and business.',
 'To help power hybrid cloud environments, and the AI that runs on them, IBM introduced the z16 platform with an integrated on-chip AI accelerator specifically designed to process and analyze real-time transactions at scale. We also unveiled IBM’s next generation LinuxONE servers, which help clients dramatically reduce data center energy consumption in support of their sustainability goals.',
 'We continued to make progress towards our goal of net-zero operational greenhouse gas emissions by 2030. In fact, we have reduced those emissions by 61% since 2010. We are using IBM Sustainability solutions to simplify and automate our sustainability reporting processes. During',
 'cloud and AI solutions. It is clear t

In [26]:
# concatenating passages
context = '\n '.join(passages)
context

'Environmental, equitable, and ethical impactBoth internally, and in our work with clients, IBM aspires to make a lasting, positive impact on the world by protecting the environment, advocating for inclusion, and fostering trust and transparency in both technology and business.\n To help power hybrid cloud environments, and the AI that runs on them, IBM introduced the z16 platform with an integrated on-chip AI accelerator specifically designed to process and analyze real-time transactions at scale. We also unveiled IBM’s next generation LinuxONE servers, which help clients dramatically reduce data center energy consumption in support of their sustainability goals.\n We continued to make progress towards our goal of net-zero operational greenhouse gas emissions by 2030. In fact, we have reduced those emissions by 61% since 2010. We are using IBM Sustainability solutions to simplify and automate our sustainability reporting processes. During\n cloud and AI solutions. It is clear that tec

## 3. Creating Prompt

In [9]:
# https://huggingface.co/blog/llama2#how-to-prompt-llama-2

prompt = \
"<s>[INST] <<SYS>> \
Please answer the following question in one sentence using this text. \
If the question is unanswerable, say 'unanswerable'. \
If you responded to the question, don't say 'unanswerable'. \
Do not include information that's not relevant to the question. \
Do not answer other questions. \
Make sure the language used is English.'\
Do not use repetitions' \
Question:" + question + \
'<</SYS>>' + context + '[/INST]'

#complete_prompt = context + instruction + question

print("----------------------------------------------------------------------------------------------------")
print("*** Prompt:" + prompt + "***")
print("----------------------------------------------------------------------------------------------------")

----------------------------------------------------------------------------------------------------
*** Prompt:<s>[INST] <<SYS>> Please answer the following question in one sentence using this text. If the question is unanswerable, say 'unanswerable'. If you responded to the question, don't say 'unanswerable'. Do not include information that's not relevant to the question. Do not answer other questions. Make sure the language used is English.'Do not use repetitions' Question:I’m interested in IBM’s effect on the environment.  What efforts have they been making in sustainability??<</SYS>>Environmental, equitable, and ethical impactBoth internally, and in our work with clients, IBM aspires to make a lasting, positive impact on the world by protecting the environment, advocating for inclusion, and fostering trust and transparency in both technology and business.
 To help power hybrid cloud environments, and the AI that runs on them, IBM introduced the z16 platform with an integrated on-c

In [27]:
def get_model(model_type,max_tokens,min_tokens,decoding,temperature):#, repetition_penalty):

    generate_params = {
        GenParams.MAX_NEW_TOKENS: max_tokens,
        GenParams.MIN_NEW_TOKENS: min_tokens,
        GenParams.DECODING_METHOD: decoding,
        GenParams.TEMPERATURE: temperature,
    }
    
    model = Model(
        model_id=model_type,
        params=generate_params,
        credentials={
            "apikey": api_key,
            "url": url
        },
        project_id=watsonx_project_id
        )

    return model

## 4. Configuring watsonx.ai

In [28]:
# URL of the hosted LLMs is hardcoded because at this time all LLMs share the same endpoint
url = "https://us-south.ml.cloud.ibm.com"

# Replace with your watsonx project id (look up in the project Manage tab)
watsonx_project_id = ""
# Replace with your IBM Cloud key
api_key = ""

In [29]:
model_type = "meta-llama/llama-2-70b-chat"
max_tokens = 100
min_tokens = 50
decoding = DecodingMethods.GREEDY
temperature = 0.7

# Get the watsonx model
model = get_model(model_type, max_tokens, min_tokens, decoding, temperature)

## 5. Answer Generation

In [30]:
generated_response = model.generate(prompt)
response_text = generated_response['results'][0]['generated_text']

# print model response
print("--------------------------------- Generated response -----------------------------------")
print(response_text)
print("*********************************************************************************************")

--------------------------------- Generated response -----------------------------------
  IBM has made efforts in sustainability by reducing their operational greenhouse gas emissions by 61% since 2010 and aiming to reach net-zero emissions by 2030, and have introduced new technology such as the z16 platform and LinuxONE servers that help clients reduce data center energy consumption.
*********************************************************************************************
